In [1]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("API_KEY")
SHOP_ID = os.getenv("SHOP_ID")

def test_page_size(shop_id, page_size):
    url = f"https://pos.pages.fm/api/v1/shops/{shop_id}/orders"
    params = {"api_key": API_KEY, "page": 1, "page_size": page_size}
    resp = requests.get(url, params=params)
    print(f"Page size {page_size} → status {resp.status_code}")
    if resp.status_code == 200:
        data = resp.json().get("data", [])
        print(f"👉 Lấy được {len(data)} bản ghi")
    else:
        print("❌ Lỗi:", resp.text[:200])

# Thử với nhiều giá trị
for ps in [50, 100, 200, 500, 1000, 2000]:
    test_page_size(SHOP_ID, ps)


Page size 50 → status 200
👉 Lấy được 50 bản ghi
Page size 100 → status 200
👉 Lấy được 100 bản ghi
Page size 200 → status 200
👉 Lấy được 200 bản ghi
Page size 500 → status 200
👉 Lấy được 500 bản ghi
Page size 1000 → status 200
👉 Lấy được 1000 bản ghi
Page size 2000 → status 422
❌ Lỗi: {"error":"page size must be less than or equal to 1000"}


In [9]:
import os
import json
import time
import random
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, VARCHAR
from sqlalchemy.types import BigInteger, DateTime
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường
load_dotenv()

API_KEY   = os.getenv("API_KEY")
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
SHOP_ID   = os.getenv("SHOP_ID")  # nhớ thêm SHOP_ID vào file .env


In [10]:
# Tạo engine tới database Bronze
engine_bronze = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}"
)

print(f"✅ Kết nối thành công tới database Bronze: {DB_BRONZE}")


✅ Kết nối thành công tới database Bronze: winner_bronze


In [11]:
def get_orders_page(shop_id, page=1, page_size=1000, max_retries=3):
    """
    Gọi API Pancake /shops/{shop_id}/orders để lấy một trang.
    Có retry + delay khi lỗi.
    """
    url = f"https://pos.pages.fm/api/v1/shops/{shop_id}/orders"
    params = {"api_key": API_KEY, "page": page, "page_size": page_size}
    
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, params=params, timeout=20)
            if resp.status_code == 200:
                j = resp.json()
                data = j.get("data", [])
                total_pages = j.get("total_pages") or j.get("meta", {}).get("total_pages")
                return data, total_pages
            else:
                print(f"⚠️ Lỗi status {resp.status_code} tại page {page}, attempt {attempt}")
        except Exception as e:
            print(f"⚠️ Exception tại page {page}: {e}, attempt {attempt}")
        time.sleep(2 * attempt)  # exponential backoff
    return [], None


In [12]:
def insert_orders_batch(df_batch):
    df_batch.to_sql(
        "orders_raw",
        con=engine_bronze,
        if_exists="append",
        index=False,
        dtype={
            "shop_id": BigInteger(),
            "order_id": VARCHAR(50),
            "raw_json": LONGTEXT(),
            "extracted_at": DateTime()
        }
    )


In [13]:
def fetch_and_load_orders(shop_id, start_page=1, page_size=1000):
    page = start_page
    while True:
        orders, total_pages = get_orders_page(shop_id, page, page_size)
        if not orders:
            print("📦 Không còn dữ liệu tại page", page)
            break

        # Tạo DataFrame batch
        df_batch = pd.DataFrame([{
            "shop_id": shop_id,
            "order_id": order.get("id"),
            "raw_json": json.dumps(order, ensure_ascii=False),
            "extracted_at": datetime.now()
        } for order in orders])

        # Insert batch vào Bronze
        insert_orders_batch(df_batch)
        print(f"✅ Page {page}/{total_pages} - Loaded {len(orders)} orders")

        if total_pages and page >= total_pages:
            break

        # Ngủ ngẫu nhiên 0.5–1.5s để tránh bị chặn
        time.sleep(random.uniform(0.5, 1.5))
        page += 1


In [14]:
fetch_and_load_orders(SHOP_ID, start_page=1, page_size=1000)


✅ Page 1/41 - Loaded 1000 orders
✅ Page 2/41 - Loaded 1000 orders
✅ Page 3/41 - Loaded 1000 orders
✅ Page 4/41 - Loaded 1000 orders
✅ Page 5/41 - Loaded 1000 orders
✅ Page 6/41 - Loaded 1000 orders
✅ Page 7/41 - Loaded 1000 orders
⚠️ Exception tại page 8: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 8/41 - Loaded 1000 orders
⚠️ Exception tại page 9: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 9/41 - Loaded 1000 orders
⚠️ Exception tại page 10: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 10/41 - Loaded 1000 orders
⚠️ Exception tại page 11: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 11/41 - Loaded 1000 orders
⚠️ Exception tại page 12: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 12/41 - Loaded 